# High level APIs
- estimator
- Experiment 

### Estimator 
- sklearn의 estimator와 유사하게 설계된 framework 
- train, evaluate, predict 등을 쉽게 구현 할 수 있음. 
- Session을 사용자가 직접 구현 할 필요가 없음 
- input pipeline만 작성하면 model 설계에 집중 할 수 있다. 
- model export 를 제공하며, Serving 시 필요한 protocol buffer file을 생성해준다. 

#### Input function  

- model에 투입되는 dataset을 load 하는 pipeline을 구성하는 함수 정의 
- 원하는 pipeline을 설계하고, 함수의 리턴은 train. labels로만 해주면 사용가능 
- Estimator에서 이 input function을 함수형으로 받기 때문에 입력 argument 필요시에 아래 코드와 같이 lambda식을 활용 

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np 

In [4]:
# estimator로 log를 남길 필요가 있다. 
tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()

# estimator에서 쓰이기 위해 input과 model의 형태를 처리 해준다. 
def parser(serialized_example):
    features = {
        'age' : tf.FixedLenFeature([1], tf.int64),
        'img' : tf.FixedLenFeature([61*49], tf.int64),        
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img

def input_fn(dataset_dir, batch_size):
    dataset = tf.data.TFRecordDataset(dataset_dir).map(parser)
    dataset = dataset.batch(batch_size).shuffle(777)
    
    itr = dataset.make_one_shot_iterator()

    age, img = itr.get_next()
    
    img = tf.reshape(img, [-1, 61, 49, 1])
    img = tf.cast(img, tf.float32)
    
    age = tf.reshape(age, [-1])
    age_onehot = tf.one_hot(age, depth=3, axis=-1)
    
    
    return {'img' : img}, {'age' : age, 'age_onehot' : age_onehot}

def get_train_input_fn():
    return lambda : input_fn('./cnn_dataset/face_train.tfrecord', 10)

def get_test_input_fn():
    return lambda : input_fn('./cnn_dataset/face_test.tfrecord', 10)


### Model function 
- 실제 모델이 들어가는 함수로써 입력인자(features, lables, mode)signature를 지켜주어야 한다. 
- 이 model function도 마찬가지로 사용자의 hyper parameters를 외부에서 제어 할 수 있도록 lambda식을 이용하여 입력인자로 넘겨 받을 수 있다. 
- 반드시 EstimatorSpec을 return 시켜주어야하며, TRAIN, EVAL, PREDICT 모드 별 트레인 모델을 분기분으로 나눠준다. 

In [5]:
def model(features, labels, mode, **params):
    conv1 = tf.layers.conv2d(features['img'], filters=10, kernel_size=3,
                            padding='SAME', activation=params['activation'])
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2)

    conv2 = tf.layers.conv2d(pool1, filters=10, kernel_size=3,
                            padding='SAME', activation=params['activation'])
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
    
    conv3 = tf.layers.conv2d(pool2, filters=10, kernel_size=3, 
                            padding="SAME", activation=params['activation'])
    pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=2)
    
    flat_size = int(pool3.shape[1]) * int(pool3.shape[2]) * int(pool3.shape[3])
    flat = tf.reshape(pool3, [-1, flat_size])
    if mode == tf.estimator.ModeKeys.TRAIN:
        dropout_prob = params['dropout_prob']
    else: 
        dropout_prob = 1
        
    dropout1 = tf.layers.dropout(flat, dropout_prob)
    fc1 = tf.layers.dense(dropout1, 1000)
    
    dropout2 = tf.layers.dropout(fc1, dropout_prob)
    out = tf.layers.dense(dropout2, 3)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.softmax_cross_entropy(labels["age_onehot"], out)
        train_op = tf.train.AdagradOptimizer(1e-2).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(train_op=train_op, loss=loss, mode=mode)
        
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss = tf.losses.softmax_cross_entropy(labels['age_onehot'], out)
        pred = tf.argmax(tf.nn.softmax(out), axis=1)
        accuracy = tf.metrics.accuracy(labels['age'], pred)
        eval_metric_ops = {"acc" : accuracy}
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
        
    else : 
        estimator_spec = None
        
    return estimator_spec


def get_model_fn(activation, dropout_prob):
    return lambda features, labels, mode:model(features, labels, mode, 
                                              activation=activation, dropout_prob=0.5)

### Estimator

- 아래와 같이 model 함수를 이용하여 Estimator를 생성해준다. 
- 생성한 Estimator를 이용하여 train, eval, predict 등을 수행 할 수 있다. 

In [6]:
est_config = tf.estimator.RunConfig()
est = tf.estimator.Estimator(model_fn=get_model_fn(activation=tf.nn.relu, dropout_prob=0.7), 
                            model_dir='./logs/estimator',
                            config=est_config)

for epoch in range(3):
    est.train(get_test_input_fn())
    est.evaluate(get_test_input_fn())

INFO:tensorflow:Using config: {'_model_dir': './logs/estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1137dac88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/estimator/model.ckpt-154
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 155 into ./logs/estimator/model

### Experiment

- estimator를 잘 돌려주는 framework
- local distributed setting 만 되어 있다면, 분산 러닝 자동 지원
- training, inference, predict 모델을 따로 저장하기 때문에 serving 시 Experiment를 통한 export권장 
- 내부에 학습 간 필요한 기능들을 다수 내장 

In [7]:
import json
import os 
def get_experiment(output_dir):
    os.environ["TF_CONFIG"] = json.dumps({
        'environment' : 'local'
    })
    config = tf.contrib.learn.RunConfig()
    return tf.contrib.learn.Experiment(estimator=tf.estimator.Estimator(
    model_fn=get_model_fn(activation=tf.nn.relu, dropout_prob=0.5),
    model_dir=output_dir,
    config=config),
                                       train_input_fn=get_train_input_fn(),
                                       eval_input_fn=get_test_input_fn())

tf.contrib.learn.learn_runner.run(experiment_fn=get_experiment, output_dir="./logs/etimator")
# tf.estimator.train_and_evaluate


Instructions for updating:
Use tf.estimator.train_and_evaluate.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1155bad68>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './logs/etimator'}
Instructions for updating:
Please switch to tf.estimator.train_and_evaluate. You will also have to convert to a tf.estimator.Estimator.
Instructions for updating:
Moni

({'acc': 0.558, 'global_step': 600, 'loss': 0.96165633}, [])